<a href="https://colab.research.google.com/github/nboravlev/Python_sample/blob/main/Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
#1

import time

def get_all_links(query):#ф-ия для получения ссылок 
  
  url = 'https://habr.com/ru/search/' #ссылка на страницу поиска 
  
  link_=[] #заранее формируем словарь, чтобы он не затирался каждый раз при выполнении цикла
  
  for t in query:#query в нашем случае это список, цикл для перебора списка
    
    params = {
        'q': t #в хабре поиск инициируется специальным атрибутом q
    }
    req = requests.get(url, params=params)#применение функции requests, запись результата в переменную req
    time.sleep(0.3)#следующий проход цикла с задержкой 0.3 сек
    soup = BeautifulSoup(req.text)#к req применим метод text и все это передаем в функцию BeautifulSoup, запись в переменнную Soup
    articles = soup.find_all('article', class_='tm-articles-list__item')#тег 'article', класс 'tm-articles-list__item'

    for article in articles:#из soup-объекта articles извлекаем ссылки. Для это цикл.
      try:#пришлось ввести try/except, одна новость кривая попалась
        link_.append(article.find('h2').find('a').get('href'))#метод finf, обращение к тэгу h2 >> a, запись в список
      except AttributeError:
        link_.append(article.find('div').find('a').get('href'))

  link_ = set(link_)#снаружи цикла список >> set для удаления дублей

  full_links = list(map(lambda x: 'https://habr.com' + x, link_))#к link_ засовываем в ф-ию map, чтобы выполнить concat и превратить ссылку в абсолютню

  return full_links #снаружи цикла выполняем return, чтобы цикл не прервался раньше времени и данные не пропали

all_links = get_all_links(['Pyhton','анализ данных'])#в кач-ве аргумента передаем список поисковых запросов и записываем результат в all_links
print(len(all_links))#собрал 40 ссылок в итоге

40


In [ ]:
def get_habr_news(links):#ф-ия для обработки ссылок и парсинга соответствующих страниц с одним параметром
    habr_news = pd.DataFrame()#создание пустого ДФ
    for link in all_links:#цикл для перебора параметра ф-ии
        soup = BeautifulSoup(requests.get(link).text)#применяется все что нужно
        time.sleep(0.33)
        date = pd.to_datetime(soup.find('time').get('datetime'))
        title = soup.find('h1').text
        row = {'date': date, 'title': title, 'link': link}#словарь, где ключ это название колонки
        habr_news = pd.concat([habr_news, pd.DataFrame([row])])#конкатенация новых строк в ДФ
    return habr_news.reset_index(drop=True)#reset_index нужен чтобы сбросить индекс, каждая новая добавленная строка имеет 0 индекс

In [ ]:
habr_news = get_habr_news(all_links)#вызов ф-ии, в кач-ве аргумента передан список ссылок
habr_news

,date,title,link
0,2014-10-05 15:57:38+00:00,Обзор наиболее интересных материалов по анализ...,https://habr.com/ru/post/239247/
1,2021-01-14 07:00:03+00:00,Что такое бессерверный SQL? И как использовать...,https://habr.com/ru/company/microsoft/blog/537...
2,2014-06-30 19:06:57+00:00,Обзор наиболее интересных материалов по анализ...,https://habr.com/ru/post/228187/
3,2013-10-08 18:52:19+00:00,Настраиваем Vim для работы с Python кодом,https://habr.com/ru/post/196550/
4,2014-09-14 15:54:01+00:00,Обзор наиболее интересных материалов по анализ...,https://habr.com/ru/post/236757/
5,2010-03-12 04:53:43+00:00,В ожидании MIX’10: ваши прогнозы,https://habr.com/ru/post/87236/
6,2021-11-22 14:58:55+00:00,Мониторинг скорости интернет каналов в Zabbix,https://habr.com/ru/post/590775/
7,2020-08-16 14:50:34+00:00,Конфуций и Маргарита,https://habr.com/ru/post/515346/
8,2010-03-09 03:22:02+00:00,Grails — священный грааль веб-разработчика,https://habr.com/ru/post/86790/
9,2014-02-24 09:03:08+00:00,"Python-digest #15. Новости, интересные проекты...",https://habr.com/ru/post/213645/


In [ ]:
habr_news.date.max() - habr_news.date.min()

Timedelta('4852 days 05:26:54')

In [ ]:
#2

In [ ]:
def get_all_links(query, pages):#ф-ия для извлечения ссылок из глубины поиска, принимает 2 параметра
    url = 'https://habr.com/ru/search/page'#это стартовая ссылка - франкенштейн
    links_list = []#всегда лучше иметь в запасе пустой словарь
    for w in query:#первый параметр ф-ии передается в цикл
      params = {
          'q': w#первый элемент списка попадает в params, спускается в следующий цикл
      }
      for i in range(1, pages+1):#цикл в цикле обрабытывает второй параметр
          res = requests.get(url+str(i), params)#конкат по очереди подставляет в урл число, таким образом проходит в глубину поиска
          soup = BeautifulSoup(res.text)#применяется метод .text, формируется объект BeautifulSoup
          time.sleep(0.33)

          articles = soup.find_all('article', class_='tm-articles-list__item')#метод find_all по определенному тэгу и классу
          all_links = []#назначается новый список внутри цикла

          for article in articles:#из soup-объекта articles извлекаем ссылки. Для этого цикл.
            try:#пришлось ввести try/except, одна новость кривая попалась
              all_links.append(article.find('h2').find('a').get('href'))#метод finf, обращение к тэгу h2 >> a, запись в список
            except AttributeError:
              all_links.append(article.find('div').find('a').get('href'))

          all_links = set(all_links)#снаружи цикла список >> set для удаления дублей

          full_links = list(map(lambda x: 'https://habr.com' + x, all_links))#all_links зпопадает в ф-ию map, чтобы выполнить concat и превратить ссылку в абсолютню

          links_list += full_links#в цикл links_list, который существует снаружи всех циклов, накапливаются все сслки

    return links_list #снаружи цикла выполняем return, чтобы цикл не прервался раньше времени и данные не пропали


list_habr = get_all_links(['java'], 2)#в кач-ве аргументов ф-ия принимает список поисковых запросов и число страниц поиска
print(len(list_habr))
print(list_habr)

40
['https://habr.com/ru/company/sberbank/news/t/676246/', 'https://habr.com/ru/company/naumen/news/t/663760/', 'https://habr.com/ru/company/naumen/news/t/687204/', 'https://habr.com/ru/company/netologyru/news/t/665874/', 'https://habr.com/ru/company/alconost/news/t/491378/', 'https://habr.com/ru/news/t/676576/', 'https://habr.com/ru/company/yandex_praktikum/news/t/702076/', 'https://habr.com/ru/company/simbirsoft/news/t/540320/', 'https://habr.com/ru/news/t/686410/', 'https://habr.com/ru/company/geekbrains/news/t/556628/', 'https://habr.com/ru/company/epam_systems/news/t/578244/', 'https://habr.com/ru/company/itsumma/news/t/490834/', 'https://habr.com/ru/news/t/689394/', 'https://habr.com/ru/news/t/522224/', 'https://habr.com/ru/company/southbridge/news/t/682064/', 'https://habr.com/ru/company/sberbank/news/t/665854/', 'https://habr.com/ru/company/deutschetelekomitsolutions/news/t/647489/', 'https://habr.com/ru/news/t/684376/', 'https://habr.com/ru/company/sberbank/news/t/646721/', 'h

In [ ]:
def get_habr_total(links):#ф-ия для обработки ссылок и парсинга соответствующих страниц с одним параметром
    habr_total = pd.DataFrame()#создание пустого ДФ
    for link in links:#цикл для перебора параметра ф-ии
        soup = BeautifulSoup(requests.get(link).text)#применяется все что нужно
        time.sleep(0.33)
        date = pd.to_datetime(soup.find('time').get('datetime'))
        try:
          title = soup.find('h1').text
          text = soup.find('div', class_='article-formatted-body').find('div').text.replace('\r','').replace('\n', '')
          likes = soup.find('span', class_='tm-votes-meter__value').text
        except AttributeError:#ошибок очень много, моя примитивная обработка не все учитывает
          title = soup.find('p', class_='tm-expired-company__text').text
          text = soup.find('p', class_='tm-expired-company__text').text
          likes = 0
        row = {'date': date, 'title': title, 'link': link, 'text': text, 'likes': int(likes)}#словарь, где ключ это название колонки
        habr_total = pd.concat([habr_total, pd.DataFrame([row])])#конкатенация новых строк в ДФ
    return habr_total.reset_index(drop=True)#reset_index нужен чтобы сбросить индекс, каждая новая добавленная строка имеет 0 индекс

In [ ]:
habr_ = get_habr_total(list_habr)#вызов ф-ии, в кач-ве аргумента передан список ссылок
habr_

,date,title,link,text,likes
0,2022-07-11 10:00:02+00:00,Ищем Java-разработчиков и Data-инженеров в ком...,https://habr.com/ru/company/sberbank/news/t/67...,Получите интересную работу без многоэтапных со...,0
1,2022-04-29 14:29:57+00:00,All You Need Is Java: анонс JUG.EKB #18,https://habr.com/ru/company/naumen/news/t/663760/,12 мая пройдет митап сообщества JUG.EKB. Присо...,0
2,2022-09-08 13:46:57+00:00,Митап для начинающих Java-разработчиков — JUG....,https://habr.com/ru/company/naumen/news/t/687204/,22 сентября пройдет онлайн-митап сообщества JU...,3
3,2022-05-16 09:26:02+00:00,Карьера Java-разработчика как проект: курсы дл...,https://habr.com/ru/company/netologyru/news/t/...,"IT — дело на всю жизнь, так что если вы собира...",-2
4,2022-02-16 09:25:01+00:00,Компания Alconost временно не ведёт блог на Хабре,https://habr.com/ru/company/alconost/news/t/49...,Компания Alconost временно не ведёт блог на Хабре,0
5,2022-07-12 16:34:44+00:00,На онлайн-платформе All Cups пройдет чемпионат...,https://habr.com/ru/news/t/676576/,Фонд «Сколково» и IT_One будут проводить чемпи...,5
6,2022-11-28 13:29:00+00:00,Вебинары от направления программирования: Java...,https://habr.com/ru/company/yandex_praktikum/n...,30 ноября Яндекс Практикум проведет вебинар на...,2
7,2021-02-01 12:35:19+00:00,Как стать разработчиком Java и С#: открываем о...,https://habr.com/ru/company/simbirsoft/news/t/...,"Какие навыки прокачать на старте, где найти ме...",0
8,2022-09-05 09:43:28+00:00,Microsoft запустила обучающий портал для разра...,https://habr.com/ru/news/t/686410/,Microsoft запустила обучающий портал для разра...,6
9,2021-05-05 13:58:01+00:00,Компания GeekBrains временно не ведёт блог на ...,https://habr.com/ru/company/geekbrains/news/t/...,Компания GeekBrains временно не ведёт блог на ...,0


In [ ]:
habr_[habr_.likes == habr_.likes.max()]

,date,title,link,text,likes
28,2015-09-14 12:20:44+00:00,Шпаргалка Java программиста 2: Триста пятьдеся...,https://habr.com/ru/post/266821/,"Что это и зачем оно надо: Как известно, главно...",35
